In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import json

import sys

sys.path.append("../")

##################################################################
os.environ["TOKENIZERS_PARALLELISM"] = "false"
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1,2,3,4,5,6,7"
##################################################################

import logging
from src.utils import logging_utils
from src.utils import env_utils

logger = logging.getLogger(__name__)

logging.basicConfig(
    level=logging.DEBUG,
    format=logging_utils.DEFAULT_FORMAT,
    datefmt=logging_utils.DEFAULT_DATEFMT,
    stream=sys.stdout,
)

import torch
import transformers

logger.info(f"{torch.__version__=}, {torch.version.cuda=}")
logger.info(
    f"{torch.cuda.is_available()=}, {torch.cuda.device_count()=}, {torch.cuda.get_device_name()=}"
)
logger.info(f"{transformers.__version__=}")

2025-06-27 04:33:27 __main__ INFO     torch.__version__='2.7.1+cu126', torch.version.cuda='12.6'
2025-06-27 04:33:27 __main__ INFO     torch.cuda.is_available()=True, torch.cuda.device_count()=8, torch.cuda.get_device_name()='NVIDIA A100 80GB PCIe'
2025-06-27 04:33:27 __main__ INFO     transformers.__version__='4.53.0'


In [7]:
from src.utils.training_utils import get_device_map

model_key = "meta-llama/Llama-3.3-70B-Instruct"

device_map = get_device_map(model_key, 32, n_gpus=8)
device_map

{'model.embed_tokens': 7,
 'model.norm': 7,
 'model.rotary_emb': 7,
 'lm_head': 7,
 'model.layers.0': 0,
 'model.layers.1': 1,
 'model.layers.2': 2,
 'model.layers.3': 3,
 'model.layers.4': 4,
 'model.layers.5': 5,
 'model.layers.6': 6,
 'model.layers.7': 7,
 'model.layers.8': 0,
 'model.layers.9': 1,
 'model.layers.10': 2,
 'model.layers.11': 3,
 'model.layers.12': 4,
 'model.layers.13': 5,
 'model.layers.14': 6,
 'model.layers.15': 7,
 'model.layers.16': 0,
 'model.layers.17': 1,
 'model.layers.18': 2,
 'model.layers.19': 3,
 'model.layers.20': 4,
 'model.layers.21': 5,
 'model.layers.22': 6,
 'model.layers.23': 7,
 'model.layers.24': 0,
 'model.layers.25': 1,
 'model.layers.26': 2,
 'model.layers.27': 3,
 'model.layers.28': 4,
 'model.layers.29': 5,
 'model.layers.30': 6,
 'model.layers.31': 7,
 'model.layers.32': 0,
 'model.layers.33': 1,
 'model.layers.34': 2,
 'model.layers.35': 3,
 'model.layers.36': 4,
 'model.layers.37': 5,
 'model.layers.38': 6,
 'model.layers.39': 7,
 'model

In [4]:
os.getcwd()

'/disk/u/gio/mechinterp'

In [8]:
from src.models import ModelandTokenizer

mt = ModelandTokenizer(
    model_key=model_key,
    torch_dtype=torch.bfloat16,
    device_map=device_map
)

Loading checkpoint shards:   0%|          | 0/30 [00:00<?, ?it/s]

2025-06-27 04:35:25 src.models INFO     loaded model <models/meta-llama/Llama-3.3-70B-Instruct> | size: 134570.516 MB | dtype: torch.bfloat16 | device: cuda:7


In [5]:
#from typing import Literal
#from src.probing.prompt import ProbingPrompt
#from src.hooking.llama_attention import AttentionEdge
#
#def get_edges_to_be_ablated(
#    prompt: ProbingPrompt,
#    focus_strategy: Literal["entities", "entities_last", "ablate_all"] = "ablate_all",
#    Q_IDX: int = -1, # almost always the last token position
#    whitelist_key_indices: list[int] = [0, -1],
#):
#    """
#    For attention ablation experiments.
#    Determines which attention connections should be blocked (ablated) between tokens.
#    Used to understand which attention patterns are important for specific model behaviors.
#
#    Creates a block list of attention edges by:
#        1. Converting negative indices to positive ones
#        2. Building a whitelist of key positions to preserve based on the strategy
#        3. Creating AttentionEdge objects for all non-whitelisted positions
#
#    Whitelisting strategy: Instead of specifying which edge to ablate, specify which connections
#        to preserve, and everything else gets ablated.
#
#    Arguments:
#        focus_strategy: controls which entity-related tokens to preserve
#            entities: keeps all tokens within both entity ranges
#            entities_last: only keeps the last token of each entity
#            ablate_all: no special entity preservation
#        Q_IDX: the query token position (usually the last token where we want to measure the effect)
#        whitelist_key_indices: additional key positions to preserve (defaults to first and last tokens)
#
#    Returns:
#        AttentionEdge: represent specific attention connections from a query token position
#            to key token positions that should be blocked.
#    """
#    for idx, ti in enumerate(whitelist_key_indices):
#        if ti < 0:
#            whitelist_key_indices[idx] = prompt.tokenized["input_ids"][0].shape[-1] + ti
#    
#    if focus_strategy == "entities":
#        whitelist_key_indices += list(range(*prompt.entity_ranges[0])) + list(
#            range(*prompt.entity_ranges[1])
#        )
#    elif focus_strategy == "entities_last":
#        whitelist_key_indices += [
#            prompt.entity_ranges[0][1] - 1,
#            prompt.entity_ranges[1][1] - 1,
#        ]
#    elif focus_strategy == "ablate_all":
#        pass
#    else:
#        raise ValueError(f"{focus_strategy=}")
#
#    whitelist_key_indices = list(set(whitelist_key_indices))
#    if Q_IDX < 0:
#        Q_IDX = prompt.tokenized["input_ids"][0].shape[-1] + Q_IDX
#
#    block_edges: list[AttentionEdge] = []
#    for k_idx in range(0, prompt.tokenized["input_ids"][0].shape[-1]):
#        if k_idx in whitelist_key_indices:
#            continue
#        block_edges.append(
#            AttentionEdge(
#                q_idx=Q_IDX,
#                k_idx=k_idx,
#            )
#        )
#
#    return block_edges
#    

In [6]:
#with open(os.path.join(env_utils.DEFAULT_DATA_DIR, "coincidences_sample.json")) as f:
#    coincidences = json.load(f)
#
#logger.info(f"{len(coincidences['examples'])=}")

In [7]:
#from typing import Literal
#from src.probing.prompt import prepare_probing_input
#from src.functional import predict_next_token
#from src.tokens import TokenizerOutput
#
#focus_strategy: Literal["entities", "entities_last", "ablate_all"] = "entities"
#
#effects = []
#
#for idx, c in enumerate(coincidences["examples"]):
#    entities = c["entity_pair"]
#    logger.info(f"({idx + 1}/{len(coincidences['examples'])})  {entities=}")
#
#    prompt = prepare_probing_input(
#        mt=mt,
#        entities=entities,
#        #prefix=prefix,
#        #answer_marker=answer_marker,
#        #question_marker=question_marker,
#        #block_separator=block_separator,
#        answer_prefix=" They are/were both",
#    )
#
#    logger.info(f"{prompt=}")
#
#    clean_answer = predict_next_token(
#        mt=mt, inputs=TokenizerOutput(data=prompt.tokenized), k=1
#    )[0][0]
#    logger.info(f"{clean_answer=}")
#
#    block_edges = get_edges_to_be_ablated(
#        prompt=prompt,
#        focus_strategy=focus_strategy,
#        Q_IDX=-2,
#        whitelist_key_indices=[0]
#    )
#
#    logger.info(f"{block_edges=}")

## Patching from a different run to check contribution

In [8]:
#from src.probing.prompt import BiAssociationPrefix
#from src.utils.experiment_utils import set_seed
#
#prefix_generator_cls = BiAssociationPrefix
#
#set_seed(9001)
#
#prefix_generator = prefix_generator_cls(
#    filter_attributes=["nationality", "profession", "school"],
#    format="_3"
#)
#
#prefix = prefix_generator.get_prefix(
#    n_valid=10,
#    n_none=1
#)
#print(prefix)

In [9]:
from src.probing.prompt import BiAssociationPrefix
from src.utils.experiment_utils import set_seed
from src.probing.few_shot_examples import (
    human_nationality,
    human_profession,
    human_alma_mater
)

prefix_generator_cls = BiAssociationPrefix

prefix_generator = prefix_generator_cls(
    filter_attributes=[
        "nationality",
        "profession",
        "school"
    ],
    format = "_3"
)

set_seed(9001)
prefix = prefix_generator.get_prefix()
print(prefix)

# Task: Find Common Attributes Between Two People
You will be given two people's names. Your job is to determine if they share ANY common attribute from the list below.

## Response Format:
- If you find a match: "Yes - [shared attribute] - [description of what they share]"
- If no match: "No - [Person 1] and [Person 2] have nothing in common"

## Attributes to Consider:
1. Same nationality → "Yes - [nationality] - they are both [nationality]"
2. Same profession → "Yes - [profession] - they are both [profession]"
3. Same school → "Yes - [school] - they both graduated from [school]"

Q: Person Y and Person Z
A: No - Person Y and Person Z have nothing in common.

Q: Person C and Person D
A: Yes - Doctor - they are both doctors.

Q: Person W and Person X
A: No - Person W and Person X have nothing in common.

Q: Person A and Person B
A: Yes - German - they are both German.

Q: Person E and Person F
A: Yes - Boston University - they both graduated from Boston University.

## Your turn, give

In [6]:
#from src.tokens import align_patching_positions, find_token_range
#
#aligned_prompts = align_patching_positions(
#    mt=mt,
#    prompt_template=sample.prompt_template,
#    clean_subj=sample.clean_entity,
#    patched_subj=sample.patched_entity,
#    trace_start_marker=prefix_generator.question_marker,
#)
#
#aligned_prompts["subj_range"]

In [7]:
import re
from typing import Optional, List
from src.dataset import ActivationPatchingSamples

def create_prompt_template(common_entity: str) -> str:
    """
    Create the prompt template with the common entity inserted.
    """
    template = f"""# Task: Find Common Attributes Between Two People
You will be given two people's names. Your job is to determine if they share ANY common attribute from the list below.

## Response Format:
- If you find a match: "Yes - [shared entity] - [description of what they share]"
- If no match: "No - [Person 1] and [Person 2] have nothing in common"

## Attributes to Consider:
1. Same nationality → "Yes - [nationality] - they are both [nationality]"
2. Same profession → "Yes - [profession] - they are both [profession]"
3. Same school → "Yes - [school] - they both graduated from [school]"

Q: Person W and Person X
A: No - Person W and Person X have nothing in common.

Q: Person E and Person F
A: Yes - Boston University - they both graduated from Boston University.

Q: Person C and Person D
A: Yes - Doctor - they are both doctors.

Q: Person A and Person B
A: Yes - German - they are both German.

## Your turn, give your answer in a single line.

Q: {{}} and {common_entity}
A: Yes -"""
    
    return template

def parse_activation_data(data_string: str) -> List[ActivationPatchingSamples]:
    """
    Parse the activation patching data from the given string format.
    
    Format: common_entity | clean_entity => " clean_answer" | <-- | patched_entity => " patched_answer"
    """
    samples = []
    
    # Split by newlines and process each line
    lines = data_string.strip().split('\n')
    
    for line in lines:
        line = line.strip()
        if not line:
            continue
            
        # Parse the line using regex
        # Pattern: entity1 | entity2 => " answer1" | <-- | entity3 => " answer2"
        pattern = r'^([^|]+)\|([^=]+)=> "([^"]+)" \| <-- \| ([^=]+)=> "([^"]+)"'
        match = re.match(pattern, line)
        
        if match:
            common_entity = match.group(1).strip()
            clean_entity = match.group(2).strip()
            clean_answer = match.group(3).strip()
            patched_entity = match.group(4).strip()
            patched_answer = match.group(5).strip()

            prompt_temp = create_prompt_template(common_entity)
            
            sample = ActivationPatchingSamples(
                prompt_template=prompt_temp,
                common_entity=common_entity,
                clean_entity=clean_entity,
                patched_entity=patched_entity,
                clean_answer=clean_answer,
                patched_answer=patched_answer,
                patched_answer_toks=mt.tokenizer.encode(patched_answer, add_special_tokens=False)
            )
            samples.append(sample)
        else:
            print(f"Warning: Could not parse line: {line}")
    
    return samples

data = f"""Grace Wanjiru | James Mwangi => " Ken" | <-- | Fatima Sheikh => " Urban"
Rahman Ali | Nasreen Begum => " Bang" | <-- | Takeshi Yamamoto => " Civil"
Takeshi Yamamoto | Yuki Tanaka => " Japanese" | <-- | Rahman Ali => " Civil"
Anna Schmidt | Hans Mueller => " German" | <-- | Yuki Tanaka => " Marketing"
Yuki Tanaka | Takeshi Yamamoto => " Japanese" | <-- | Anna Schmidt => " Marketing"
Ayse Kaya | Maria Santos => " Doctor" | <-- | Hans Mueller => " Economist"
Hans Mueller | Anna Schmidt => " German" | <-- | Ayse Kaya => " Economist"
Priya Patel | Rajesh Kumar => " Indian" | <-- | Sofia Hernandez => " Graphic"
Sofia Hernandez | Carlos Rodriguez => " Mexican" | <-- | Priya Patel => " Graphic"
Zahra Hosseini | Ali Rezaei => " Iranian" | <-- | Ahmed Hassan => " Pilot"
Ahmed Hassan | Layla Mahmoud => " Egyptian" | <-- | Zahra Hosseini => " Pilot"
Kwame Mensah | Rahman Ali => " Doctor" | <-- | Min-jun Park => " Ge"
Tran Thi Mai | Nguyen Van Duc => " Vietnamese" | <-- | Marco Rossi => " Architect"
Marco Rossi | Giulia Romano => " Italian" | <-- | Tran Thi Mai => " Architect"
Giulia Romano | Marco Rossi => " Italian" | <-- | Akosua Boateng => " Data"
Alexandru Popescu | Elena Ionescu => " Romanian" | <-- | David Thompson => " Environmental"
David Thompson | Sarah MacDonald => " McGill" | <-- | Alexandru Popescu => " Environmental"
António Costa | Isabel Ferreira => " Portuguese" | <-- | Jack Wilson => " Software"
Astrid Lindgren | Erik Andersson => " Swedish" | <-- | Lisa van der Berg => " Web"
Rodrigo Gonzalez | Camila Torres => " Chile" | <-- | Maria dela Rosa => " Mechanical"
"""

# Parse the data
samples = parse_activation_data(data)

# Print parsed samples
print(f"Parsed {len(samples)} samples:\n")
for i, sample in enumerate(samples[:5]):  # Show first 5
    print(f"Sample {i+1}:")
    print(f"  Prompt Template: {sample.prompt_template}")
    print(f"  Common Entity: {sample.common_entity}")
    print(f"  Clean Entity: {sample.clean_entity}")
    print(f"  Clean Answer: {sample.clean_answer}")
    print(f"  Patched Entity: {sample.patched_entity}")
    print(f"  Patched Answer: {sample.patched_answer}")
    print(f"  Patched Answer Token: {sample.patched_answer_toks}")
    print(f"  String representation: {sample}")
    print()

# Convert to JSON
print("\nJSON representation of first sample:")
print(samples[0].to_json(indent=2))

Parsed 20 samples:

Sample 1:
  Prompt Template: # Task: Find Common Attributes Between Two People
You will be given two people's names. Your job is to determine if they share ANY common attribute from the list below.

## Response Format:
- If you find a match: "Yes - [shared entity] - [description of what they share]"
- If no match: "No - [Person 1] and [Person 2] have nothing in common"

## Attributes to Consider:
1. Same nationality → "Yes - [nationality] - they are both [nationality]"
2. Same profession → "Yes - [profession] - they are both [profession]"
3. Same school → "Yes - [school] - they both graduated from [school]"

Q: Person W and Person X
A: No - Person W and Person X have nothing in common.

Q: Person E and Person F
A: Yes - Boston University - they both graduated from Boston University.

Q: Person C and Person D
A: Yes - Doctor - they are both doctors.

Q: Person A and Person B
A: Yes - German - they are both German.

## Your turn, give your answer in a single line.

Q:

In [8]:
print(str(samples[0]))

Grace Wanjiru | James Mwangi => "Ken" | <-- | Fatima Sheikh => "Urban"


In [9]:
sample_idx = 3
sample = samples[sample_idx]
print(sample)

Anna Schmidt | Hans Mueller => "German" | <-- | Yuki Tanaka => "Marketing"


In [10]:
from src.functional import generate_with_patch

mt.reset_forward()

patch_prompt = sample.prompt_template.format(sample.patched_entity)
answer = generate_with_patch(
    mt = mt,
    inputs = patch_prompt,
    do_sample=False,
    n_gen_per_prompt=1,
    remove_prefix=True
)[0]


print(f"{(sample.patched_entity, sample.common_entity)} >> {answer=}")

keywords = sample.patched_answer_toks
print(f"{[f'{t}({mt.tokenizer.decode(t)})' for t in keywords]}")

clean_prompt = sample.prompt_template.format(sample.clean_entity)
clean_answer = generate_with_patch(
    mt=mt,
    inputs=clean_prompt,
    do_sample=False,
    n_gen_per_prompt=1,
    remove_prefix=True
)[0]
print(f"{(sample.clean_entity, sample.common_entity)} >> {clean_answer=}")

The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


('Yuki Tanaka', 'Anna Schmidt') >> answer=' Japanese - they are both Japanese.'
['68662(Marketing)']
('Hans Mueller', 'Anna Schmidt') >> clean_answer=' German - they are both German.'


In [11]:
from src.probing.prompt import BiAssociationPrefix
from src.utils.experiment_utils import set_seed
from src.probing.few_shot_examples import (
    human_nationality,
    human_profession,
    human_alma_mater,
    human_allergy,
    human_car,
    human_pet
)

prefix_generator_cls = BiAssociationPrefix

prefix_generator = prefix_generator_cls(
    filter_attributes=[
        "nationality",
        "profession",
        "school"
    ],
    format = "_3"
)

set_seed(9001)
prefix = prefix_generator.get_prefix()
print(prefix)

# Task: Find Common Attributes Between Two People
You will be given two people's names. Your job is to determine if they share ANY common attribute from the list below.

## Response Format:
- If you find a match: "Yes - [shared attribute] - [description of what they share]"
- If no match: "No - [Person 1] and [Person 2] have nothing in common"

## Attributes to Consider:
1. Same nationality → "Yes - [nationality] - they are both [nationality]"
2. Same profession → "Yes - [profession] - they are both [profession]"
3. Same school → "Yes - [school] - they both graduated from [school]"

Q: Person Y and Person Z
A: No - Person Y and Person Z have nothing in common.

Q: Person C and Person D
A: Yes - Doctor - they are both doctors.

Q: Person W and Person X
A: No - Person W and Person X have nothing in common.

Q: Person A and Person B
A: Yes - German - they are both German.

Q: Person E and Person F
A: Yes - Boston University - they both graduated from Boston University.

## Your turn, give

In [12]:
from src.tokens import align_patching_positions

aligned_prompts = align_patching_positions(
    mt=mt,
    prompt_template=sample.prompt_template,
    clean_subj=sample.clean_entity,
    patched_subj=sample.patched_entity,
    trace_start_marker=prefix_generator.question_marker
)

aligned_prompts["subj_range"]

trace_start_idx=245


(245, 249)

In [13]:
from src.tokens import prepare_input, find_token_range
from src.utils.typing import TokenizerOutput

text = mt.tokenizer.decode(
    aligned_prompts["clean_input"]["input_ids"][0], skip_special_tokens=False
)

clean_inputs = prepare_input(
    prompts=text, tokenizer=mt, add_special_tokens=False, return_offsets_mapping=True
)

assert torch.allclose(
    clean_inputs.input_ids, aligned_prompts["clean_input"]["input_ids"]
)

print(f"{sample.common_entity=}")

common_entity_range = find_token_range(
    string=text,
    substring=sample.common_entity,
    tokenizer=mt,
    offset_mapping=clean_inputs.offset_mapping[0],
    add_special_tokens=False,
)

print(f"{common_entity_range=}")

tokenized_input = TokenizerOutput(data=aligned_prompts["clean_input"])
print(
    f'"{mt.tokenizer.decode(tokenized_input.input_ids[0][common_entity_range[0] : common_entity_range[1]])}"'
)

sample.common_entity='Anna Schmidt'
common_entity_range=(250, 252)
" Anna Schmidt"


In [14]:
from src.probing.utils import ProbingPrompt

aligned_clean_prompt = ProbingPrompt(
    prompt=mt.tokenizer.decode(
        aligned_prompts["clean_input"]["input_ids"][0], skip_special_tokens=False
    ),
    entities=[sample.clean_entity, sample.common_entity],
    model_key=model_key,
    tokenized=aligned_prompts["clean_input"],
    entity_ranges=[
        aligned_prompts["subj_range"],
        common_entity_range,
    ],
    query_range=[-1, -1]
)

print(
    f'{sample.clean_entity=} | "{mt.tokenizer.decode(aligned_clean_prompt.tokenized.input_ids[0][aligned_prompts["subj_range"][0] : aligned_prompts["subj_range"][1]])}"'
)
print(
    f'{sample.common_entity=} | "{mt.tokenizer.decode(aligned_clean_prompt.tokenized.input_ids[0][common_entity_range[0] : common_entity_range[1]])}"'
)

aligned_patch_prompt = ProbingPrompt(
    prompt=mt.tokenizer.decode(
        aligned_prompts["patched_input"]["input_ids"][0], skip_special_tokens=False
    ),
    entities=[sample.patched_entity, sample.common_entity],
    model_key=model_key,
    tokenized=aligned_prompts["patched_input"],
    entity_ranges=[
        aligned_prompts["subj_range"],
        common_entity_range,
    ],
    query_range=[-1, -1],
)

print(
    f'{sample.patched_entity=} | "{mt.tokenizer.decode(aligned_patch_prompt.tokenized.input_ids[0][aligned_prompts["subj_range"][0] : aligned_prompts["subj_range"][1]])}'
)
print(
    f'{sample.common_entity=} | "{mt.tokenizer.decode(aligned_patch_prompt.tokenized.input_ids[0][common_entity_range[0] : common_entity_range[1]])}"'
)

sample.clean_entity='Hans Mueller' | "<|eot_id|><|eot_id|> Hans Mueller"
sample.common_entity='Anna Schmidt' | " Anna Schmidt"
sample.patched_entity='Yuki Tanaka' | " Yuki Tanaka
sample.common_entity='Anna Schmidt' | " Anna Schmidt"


In [18]:
PATCH_LAYER_WINDOW = list(range(5, 10))
METRIC = "logit"

In [34]:
from src.functional import interpret_logits, get_hs
from src.functional import PatchSpec
from src.trace import get_score
from typing import Literal

@torch.inference_mode()
def patched_run(
    mt: ModelandTokenizer,
    inputs: TokenizerOutput,
    patches: list[PatchSpec],
    ans_tokens: list[int],
    metric: Literal["logit", "prob"] = "logit",
    generate_full_ans: bool = False,
    **next_tok_kwargs,
):
    if generate_full_ans:
        answer = generate_with_patch(
            mt=mt,
            inputs=inputs,
            n_gen_per_prompt=1,
            do_sample=False,
            patches=patches,
            patch_strategy="replace",
            remove_prefix=True,
            patch_at_all_generations=False
        )
        print(f'"{answer[0]}"')

    logits = get_hs(
        mt=mt,
        input=inputs,
        locations=[(mt.lm_head_name, -1)],
        patches=patches,
        return_dict=False,
    ).squeeze()

    pred, track = interpret_logits(
        tokenizer=mt, logits=logits, interested_tokens=ans_tokens, **next_tok_kwargs
    )

    score = get_score(logits=logits, token_id=ans_tokens, metric=metric)

    return score, pred, track

In [47]:
import itertools
from src.functional import get_hs
from src.utils.typing import TokenizerOutput

patch_states = get_hs(
    mt=mt,
    input=TokenizerOutput(data=aligned_prompts["patched_input"]),
    locations=list(
        itertools.product(mt.layer_names, list(range(*aligned_prompts["subj_range"])))
    ),
    return_dict=True
)

gold_score, gold_pred, gold_track = patched_run(
    mt=mt,
    inputs=TokenizerOutput(data=aligned_prompts["patched_input"]),
    patches=[],
    ans_tokens=keywords,
    metric=METRIC,
)
print(f"{gold_score=}")
print(f"{gold_pred=}")
print(f"{gold_track=}")

corrupt_score, corrupt_pred, corrupt_track = patched_run(
    mt=mt,
    # inputs = TokenizerOutput(data = clean_prompt.tokenized),
    inputs=TokenizerOutput(data=aligned_prompts["clean_input"]),
    patches=[],
    ans_tokens=keywords,
    # generate_full_ans = True,
    metric=METRIC,
)
print(f"{corrupt_score=}")
print(f"{corrupt_pred=}")
print(f"{corrupt_track=}")

layer_window = [mt.layer_name_format.format(l) for l in PATCH_LAYER_WINDOW]

print(f"{layer_window=}")

tokenized_input = TokenizerOutput(data=aligned_prompts["clean_input"])

patches = [
    PatchSpec(location=(layer, token_idx), patch=patch_states[(layer, token_idx)])
    for layer, token_idx in itertools.product(
        layer_window, range(*aligned_prompts["subj_range"])
    )
]

print(f"{patches=}")

patched_score, pred, track = patched_run(
    mt=mt,
    inputs=tokenized_input,
    patches=patches,
    ans_tokens=keywords,
    generate_full_ans=True,
    metric=METRIC,
)

print(f"{patched_score=}")
print(f"{pred=}")
print(f"{track=}")

indirect_effect = (patched_score - corrupt_score) / (gold_score - corrupt_score)
print(f"{indirect_effect=}")

The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


gold_score=1.5078125
gold_pred=[PredictedToken(token=' Japanese', prob=0.37109375, logit=19.875, token_id=11002, metadata=None), PredictedToken(token=' German', prob=0.09375, logit=18.5, token_id=6063, metadata=None), PredictedToken(token=' Both', prob=0.0830078125, logit=18.375, token_id=11995, metadata=None), PredictedToken(token=' They', prob=0.0732421875, logit=18.25, token_id=2435, metadata=None), PredictedToken(token=' Same', prob=0.056884765625, logit=18.0, token_id=26823, metadata=None)]
gold_track={68662: (62484, PredictedToken(token='Marketing', prob=3.92901711165905e-09, logit=1.5078125, token_id=68662, metadata=None))}
corrupt_score=0.9375
corrupt_pred=[PredictedToken(token=' German', prob=0.7109375, logit=20.5, token_id=6063, metadata=None), PredictedToken(token=' Germans', prob=0.03125, logit=17.375, token_id=42037, metadata=None), PredictedToken(token=' Hans', prob=0.027587890625, logit=17.25, token_id=25844, metadata=None), PredictedToken(token=' Both', prob=0.024414062

In [43]:
from src.functional import predict_next_token

predict_next_token(
    mt = mt,
    inputs  =TokenizerOutput(data=aligned_prompts["clean_input"])
)

[[PredictedToken(token=' German', prob=0.7109375, logit=20.5, token_id=6063, metadata=None),
  PredictedToken(token=' Germans', prob=0.03125, logit=17.375, token_id=42037, metadata=None),
  PredictedToken(token=' Hans', prob=0.027587890625, logit=17.25, token_id=25844, metadata=None),
  PredictedToken(token=' Both', prob=0.0244140625, logit=17.125, token_id=11995, metadata=None),
  PredictedToken(token=' Germany', prob=0.021484375, logit=17.0, token_id=10057, metadata=None)]]

In [45]:
from src.attention import get_attention_matrices

tokenized_input = TokenizerOutput(data=aligned_prompts["patched_input"])

print(tokenized_input.input_ids.shape)

attn_output = get_attention_matrices(
    input = tokenized_input,
    mt = mt,
)

score = get_score(
    logits = attn_output.logits,
    token_id = keywords,
    metric = METRIC
)

print(f"{score=}")

indirect_effect = (score - corrupt_score) / (gold_score - corrupt_score)
print(f"{indirect_effect=}")

interpret_logits(logits = attn_output.logits, tokenizer=mt, interested_tokens=keywords)

torch.Size([1, 257])
output.keys(: odict_keys(['last_hidden_state', 'past_key_values', 'attentions'])
logits.shape: torch.Size([128256])
score=1.5703125
indirect_effect=1.1095890410958904


([PredictedToken(token=' Japanese', prob=0.42578125, logit=20.0, token_id=11002, metadata=None),
  PredictedToken(token=' German', prob=0.083984375, logit=18.375, token_id=6063, metadata=None),
  PredictedToken(token=' Both', prob=0.07421875, logit=18.25, token_id=11995, metadata=None),
  PredictedToken(token=' They', prob=0.0576171875, logit=18.0, token_id=2435, metadata=None),
  PredictedToken(token=' Same', prob=0.051025390625, logit=17.875, token_id=26823, metadata=None)],
 {68662: (59095,
   PredictedToken(token='Marketing', prob=4.220055416226387e-09, logit=1.5703125, token_id=68662, metadata=None))})

In [46]:
from src.attention import visualize_average_attn_matrix

visualize_average_attn_matrix(
    mt = mt,
    attn_matrices = attn_output,
    prompt=aligned_patch_prompt,
    layer_window=range(5, 14),
    q_index=-1,
    remove_bos=True,
    start_from = aligned_prompts["subj_range"][0]
)

layer=5


--------------------------------------------------------------------------------
layer=6


--------------------------------------------------------------------------------
layer=7


--------------------------------------------------------------------------------
layer=8


--------------------------------------------------------------------------------
layer=9


--------------------------------------------------------------------------------
layer=10


--------------------------------------------------------------------------------
layer=11


--------------------------------------------------------------------------------
layer=12


--------------------------------------------------------------------------------
layer=13


--------------------------------------------------------------------------------


### Patching while ablating heads

In [48]:
ATTN_LAYER_WINDOW = list(range(30, 40))

In [49]:
tokenized_input = TokenizerOutput(data=aligned_prompts["clean_input"])

start_idx = aligned_prompts["trace_start_idx"]
context_range = list(range(0, start_idx + 1))

In [ ]:
# blocking attention to the subject tokens
import baukit

mt.reset_forward()

focus_edges_on = get_edges_to_be_ablated(
    prompt=aligned_clean_prompt,
    focus_strategy="entities",
    Q_IDX=-1,
    whitelist_key_indices=context_range + [-1],
)